In [5]:
import numpy as np
import mindspore
import mindspore.dataset as ds
import mindspore.ops as ops
from mindspore import Tensor
from d2lkewei import mindspore as d2l
import mindspore as ms

true_w = ms.Tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

In [6]:
class SyntheticData():
    def __init__(self):
        self.features, self.labels = d2l.synthetic_data(true_w, true_b, 1000)

    def __getitem__(self, index):
        return self.features[index], self.labels[index]
    
    def __len__(self):
        return len(self.labels)

def load_array(data_arrays, column_names, batch_size, is_train=True):  
    """构造一个MindSpore数据迭代器。"""
    dataset = ds.GeneratorDataset(data_arrays, column_names, shuffle=is_train)
    dataset = dataset.batch(batch_size)
    return dataset

batch_size = 10
data_iter = SyntheticData()
dataset = load_array(data_iter, ['features', 'labels'], batch_size)

In [7]:
next(iter(dataset))

RuntimeError: Exception thrown from user defined Python function in dataset. 

------------------------------------------------------------------
- Python Call Stack: 
------------------------------------------------------------------
Traceback (most recent call last):
  File "/root/anaconda3/envs/MindSpore/lib/python3.9/site-packages/mindspore/dataset/engine/datasets_user_defined.py", line 99, in _cpp_sampler_fn
    val = dataset[i]
  File "/tmp/ipykernel_628798/2320612087.py", line 6, in __getitem__
    return self.features[index], self.labels[index]
  File "/root/anaconda3/envs/MindSpore/lib/python3.9/site-packages/mindspore/common/tensor.py", line 413, in __getitem__
    out = tensor_operator_registry.get('__getitem__')(self, index)
  File "/root/anaconda3/envs/MindSpore/lib/python3.9/site-packages/mindspore/ops/composite/multitype_ops/_compile_utils.py", line 180, in _tensor_getitem
    new_index, tensor_update_types, tensor_update_args = getitem_tensor_index_info(
  File "/root/a

In [4]:
# nn是神经网络的缩写
from mindspore import nn
from mindspore.common.initializer import initializer, Normal

net = nn.SequentialCell([nn.Dense(2, 1)])

In [5]:
net[0].weight = initializer(Normal(), net[0].weight.shape, mindspore.float32)
net[0].bias = initializer('zero', net[0].bias.shape, mindspore.float32)

In [6]:
loss = nn.MSELoss()

In [7]:
optimizer = nn.SGD(net.trainable_params(), learning_rate=0.03)

In [8]:
# 构造前向网络
def forward_fn(x, y):
    y_hat = net(x)
    l = loss(y_hat, y)
    return l
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in dataset:
        grad_fn = mindspore.value_and_grad(forward_fn, grad_position=None, weights=optimizer.parameters)
        l, grads = grad_fn(X, y)
        optimizer(grads)
    l = forward_fn(mindspore.Tensor(data_iter.features), mindspore.Tensor(data_iter.labels))
    print(f'epoch {epoch + 1}, loss {l.asnumpy():f}')

epoch 1, loss 0.000214
epoch 2, loss 0.000108
epoch 3, loss 0.000108


In [9]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

w的估计误差： [Tensor(shape=[], dtype=Float32, value= -0.00017786)
 Tensor(shape=[], dtype=Float32, value= -0.000656366)]
b的估计误差： [-0.00027466]
